# Generelt om saksbehandling i pesys

In [ ]:
import cx_Oracle
import pandas as pd
import numpy as np
import sys
import os
import importlib
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from google.cloud import secretmanager

sys.path.append('../lib')
import pandas_utils
import pesys_utils
from autograd_plot import nav_colors

importlib.reload(pandas_utils)
importlib.reload(pesys_utils)

In [ ]:
secrets = secretmanager.SecretManagerServiceClient()
resource_name = f"projects/knada-gcp/secrets/vebjorns-hemmelighet/versions/latest"
secret = secrets.access_secret_version(name=resource_name)
secrets = dict([line.split(":") for line in secret.payload.data.decode('UTF-8').splitlines()])

ORACLE_HOST = '10.53.136.15'
ORACLE_PORT = '1521'
ORACLE_SERVICE = 'pen'

In [ ]:
dsnStr = cx_Oracle.makedsn(ORACLE_HOST, ORACLE_PORT, service_name=ORACLE_SERVICE)

In [ ]:
con = cx_Oracle.connect(user=secrets["PEN_USER"], password=secrets["PEN_PASSWORD"],
                               dsn=dsnStr)

In [ ]:
df_v4 = pandas_utils.pandas_from_sql('../sql/v4.sql', con)

In [ ]:
k_krav_gjelder = pandas_utils.pandas_from_sql('../sql/k_krav_gjelder.sql', con)
k_krav_gjelder = dict(zip(k_krav_gjelder.K_KRAV_GJELDER, k_krav_gjelder.DEKODE))

In [ ]:
con.close()

In [ ]:
df = df_v4.copy()
# Add "ÅR-MÅNED" TO FRONT
df["ÅR-MÅNED"] = df[["ÅR", "MÅNED"]].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
cols = list(df.columns.values)
cols.insert(0, cols.pop())
df = df[cols]

df["STØNADSOMRÅDE"] = df.STØNADSOMRÅDE.apply(pesys_utils.map_stonad)
sjeldne_stonader = ["Krigspensjon", "Familiepleierytelse", "Gammel yrkesskade", "Generell"]

In [ ]:
def map_k_krav_gjelder(kode):
    return k_krav_gjelder[kode]

In [ ]:
df["SAKSTYPE"] = df.SAKSTYPE.apply(lambda kode: map_k_krav_gjelder(kode))

In [ ]:
df.head()

In [ ]:
for stonad in sjeldne_stonader:
    df = df.loc[df.STØNADSOMRÅDE != stonad]

In [ ]:
pastel = px.colors.qualitative.Pastel
figs = {}

# Saker i alle ytelser

## Uten batch

In [ ]:
df_ingen_batch = df.loc[df.BATCH_FLAGG == 'ikke opprettet av batch']

df_ingen_batch_stonad = df_ingen_batch.groupby(["ÅR-MÅNED", "STØNADSOMRÅDE"], as_index=False).ANTALL.sum().sort_values(["ÅR-MÅNED", "ANTALL"], ascending=[True, False])

fig = px.bar(df_ingen_batch_stonad, "ÅR-MÅNED", "ANTALL", color="STØNADSOMRÅDE", color_discrete_sequence=nav_colors, log_y=False)
figs["saker_totalt_ib"] = fig

## Bare batch

In [ ]:
df_batch = df.loc[df.BATCH_FLAGG == 'opprettet av batch']

df_batch_stonad = df_batch.groupby(["ÅR-MÅNED", "STØNADSOMRÅDE"], as_index=False).ANTALL.sum().sort_values(["ÅR-MÅNED", "ANTALL"], ascending=[True, False])

fig = px.bar(df_batch_stonad, "ÅR-MÅNED", "ANTALL", color="STØNADSOMRÅDE", color_discrete_sequence=nav_colors, log_y=True)
figs["saker_totalt_b"] = fig

# Sakstyper

In [ ]:
df_sakstype = df.groupby(["ÅR-MÅNED", "SAKSTYPE"], as_index=False).ANTALL.sum().sort_values(["ÅR-MÅNED", "ANTALL"], ascending=[True, False])

In [ ]:
px.bar(df_sakstype, "ÅR-MÅNED", "ANTALL", color="SAKSTYPE", log_y=True)

# Sjekk figurer

In [ ]:
for _, fig in figs.items():
    fig.show()